In [2]:
import numpy as np
import tensorflow as tf
from curriculum_learning.models.classifier_model import ClassifierModel
from curriculum_learning import utils
import yaml
from sklearn.model_selection import train_test_split

In [3]:
with open("models_hyperparameters.yaml", "r") as stream:
    models_hyperparameters = yaml.safe_load(stream)

In [4]:
N_EPOCHS = 50
BATCH_SIZE = 512

loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [5]:
x1, y1 = utils.load_cifar_data("../data/cifar-10-batches-py/")
x1 /= 255

x, _, y, _ = train_test_split(x1, y1, train_size=0.4, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.1, random_state=42)

n_classes = len(np.unique(y))
train_size = x_train.shape[0]

In [6]:
train_size

16000

In [7]:
assessment_model = ClassifierModel(output_shape=n_classes, **models_hyperparameters["assessment_model"])

assessment_model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])

history = assessment_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=N_EPOCHS, batch_size=BATCH_SIZE)

Epoch 1/50


2024-03-24 14:39:18.326745: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2024-03-24 14:39:18.326770: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-03-24 14:39:18.326775: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-03-24 14:39:18.326792: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-24 14:39:18.326801: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-03-24 14:39:19.317398: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 64ms/step - accuracy: 0.1722 - loss: 2.3628 - val_accuracy: 0.2050 - val_loss: 2.2590
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.2805 - loss: 1.9916 - val_accuracy: 0.1300 - val_loss: 2.2753
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.3519 - loss: 1.8110 - val_accuracy: 0.1150 - val_loss: 2.3438
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.3852 - loss: 1.6983 - val_accuracy: 0.1075 - val_loss: 2.4505
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.4219 - loss: 1.5879 - val_accuracy: 0.1350 - val_loss: 2.5021
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.4509 - loss: 1.5156 - val_accuracy: 0.1400 - val_loss: 2.5086
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.4750 - loss: 1.4500 - val_accuracy: 0.1925 - val_loss: 2.5265
Epoch 8/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.4879 - loss: 1.4003 - val_accuracy: 0.1900 - val_loss: 2.

In [8]:
assessment_model.save_weights('../models/assessment_model.weights.h5')